## **EVALUATING THE PERFORMANCE OF POPULAR NEURAL NETWORK ARCHITECTURES ON THE CIFAR-10 DATASET - Code File**

### Gagandip Chane
### Mathusan Thanabalasingam

In [ ]:
import numpy as np

import keras
import tensorflow.keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import models, layers, optimizers, regularizers
from keras.utils import np_utils
from tensorflow.keras.models import Sequential, Model
import matplotlib.pyplot as plt
import skimage.transform
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import * 
from tensorflow.keras.applications import InceptionV3, VGG16, ResNet50

In [ ]:
# Load dataset as train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Set numeric type to float32 from uint8
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Transform labels to one-hot encoding
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

print(x_train.shape)
print(x_test.shape)

In [ ]:
# testing with imagedatagen to see if there is any improvement:
datagen = ImageDataGenerator(
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images

datagen.fit(x_train)

# **LeNet Implementation** 

In [ ]:
LeNetmodel = Sequential()

LeNetmodel.add(layers.Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=(32,32,3), padding="same"))
LeNetmodel.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding="valid"))

LeNetmodel.add(layers.Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding="valid"))
LeNetmodel.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding="valid"))

#Flatten the CNN output so that we can connect it with fully connected layers
LeNetmodel.add(layers.Flatten())

LeNetmodel.add(Dense(120, activation='tanh'))
LeNetmodel.add(Dense(84, activation='tanh'))
LeNetmodel.add(Dense(10, activation="softmax"))

# Compile the model
LeNetmodel.compile(loss= 'categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [ ]:
LeNetmodel.summary()

In [ ]:
datagen_LeNet = LeNetmodel.fit(datagen.flow(x_train, y_train, batch_size=128),
                        steps_per_epoch=len(x_train) / 128, epochs = 50, validation_data=(x_test, y_test), verbose = 1)

In [ ]:
f, ax = plt.subplots()
ax.plot([None] + datagen_LeNet.history['accuracy'], 'o-')
ax.plot([None] + datagen_LeNet.history['val_accuracy'], 'x-')
# Plot legend and use the best location automatically: loc = 0.
ax.legend(['Train acc', 'Validation acc'], loc = 0)
ax.set_title('Training/Validation acc per Epoch')
ax.set_xlabel('Epoch')
ax.set_ylabel('acc')

# **Keras VGG-16 (Transfer Learning)**

In [ ]:
# keras.applications implementations of models
input_shape = (32, 32, 1)
X_input = Input(input_shape)

class Gray2VGGInput( Layer ) :
    """Custom conversion layer
    """
    def build( self, x ) :
        self.image_mean = K.variable(value=np.array([103.939, 116.779, 123.68]).reshape([1,1,1,3]).astype('float32'), 
                                     dtype='float32', 
                                     name='imageNet_mean' )
        self.built = True
        return
    def call( self, x ) :
        rgb_x = K.concatenate( [x,x,x], axis=-1 )
        norm_x = rgb_x - self.image_mean
        return norm_x
    def compute_output_shape( self, input_shape ) :
        return input_shape[:3] + (3,)

VGG16tf = Sequential()
#new_VGG16.add(VGG16(include_top = False, weights = None, input_tensor = X_input, input_shape = input_shape))
VGG16tf.add(VGG16(include_top = False, weights = 'imagenet', input_tensor = X_input, input_shape = input_shape))
VGG16tf.add(Flatten())
VGG16tf.add(layers.Dense(2048, activation='relu', kernel_regularizer=regularizers.l2(0.0005)))
VGG16tf.add(layers.BatchNormalization())
VGG16tf.add(layers.Dense(2048, activation='relu', kernel_regularizer=regularizers.l2(0.0005)))
VGG16tf.add(layers.BatchNormalization())
VGG16tf.add(Dense(10, activation = 'softmax'))

#specify that the VGG16 layers should not be adjusted
VGG16tf.layers[0].trainable = False

VGG16tf.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
VGG16tf.summary()

In [ ]:
plot_pretrained_VGG = VGG16tf.fit(datagen.flow(x_train, y_train, batch_size=128), 
                                              steps_per_epoch=len(x_train) / 128, epochs = 50, validation_data=(x_test, y_test), verbose = 1)

In [ ]:
f1, ax1 = plt.subplots()
ax1.plot([None] + plot_pretrained_VGG.history['accuracy'], 'o-')
ax1.plot([None] + plot_pretrained_VGG.history['val_accuracy'], 'x-')
# Plot legend and use the best location automatically: loc = 0.
ax1.legend(['Train acc', 'Validation acc'], loc = 0)
ax1.set_title('Training/Validation acc per Epoch')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('acc')

# **Keras VGG-16 (No Transfer Learning)**

In [ ]:
# keras.applications implementations of models
input_shape = (32, 32, 3)
X_input = Input(input_shape)

VGG16no_tf = Sequential()
VGG16no_tf.add(VGG16(include_top = False, weights = None, input_tensor = X_input, input_shape = input_shape))
VGG16no_tf.add(Flatten())
VGG16no_tf.add(layers.Dense(2048, activation='relu', kernel_regularizer=regularizers.l2(0.0005)))
VGG16no_tf.add(layers.BatchNormalization())
VGG16no_tf.add(layers.Dense(2048, activation='relu', kernel_regularizer=regularizers.l2(0.0005)))
VGG16no_tf.add(layers.BatchNormalization())
VGG16no_tf.add(Dense(10, activation = 'softmax'))

VGG16no_tf.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
VGG16no_tf.summary()

In [ ]:
plot_untrained_VGG = VGG16no_tf.fit(datagen.flow(x_train, y_train, batch_size=128), 
                                              steps_per_epoch=len(x_train) / 128, epochs = 50, validation_data=(x_test, y_test), verbose = 1)

In [ ]:
f1, ax1 = plt.subplots()
ax1.plot([None] + plot_untrained_VGG.history['accuracy'], 'o-')
ax1.plot([None] + plot_untrained_VGG.history['val_accuracy'], 'x-')
# Plot legend and use the best location automatically: loc = 0.
ax1.legend(['Train acc', 'Validation acc'], loc = 0)
ax1.set_title('Training/Validation acc per Epoch')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('acc')

# **Keras Inception V3 (No Transfer Learning)**

In [ ]:
# CODE BLOCK REQUIRED TO RESIZE IMAGES

# for x_train:
newImages = []

for image in x_train:
  newImage = skimage.transform.resize(image, (75, 75, 3), mode='constant')
  newImages.append(newImage)

newImages = np.stack(newImages, axis=0)

inception_xtrain = newImages

# for x_test:
newImages = []

for image in x_test:
  newImage = skimage.transform.resize(image, (75, 75, 3), mode='constant')
  newImages.append(newImage)

newImages = np.stack(newImages, axis=0)

inception_xtest = newImages

In [ ]:
InceptionDatagen = ImageDataGenerator(
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images

In [ ]:
# compute quantities required for featurewise normalization
InceptionDatagen.fit(inception_xtrain)

In [ ]:
input_shape = (75, 75, 3)
X_input = Input(input_shape)

inceptionModel = Sequential()
inceptionModel.add(InceptionV3(include_top = False, weights = None, input_tensor = X_input, input_shape = input_shape))
inceptionModel.add(GlobalAveragePooling2D())
inceptionModel.add(Dense(10, activation = 'softmax'))
inceptionModel.compile(optimizer= 'adam', loss='categorical_crossentropy', metrics=['accuracy'])
inceptionModel.summary()

In [ ]:
plot_Inception = inceptionModel.fit(InceptionDatagen.flow(inception_xtrain, y_train, batch_size=128),
                        steps_per_epoch=len(x_train) / 128, epochs = 20, validation_data=(inception_xtest, y_test), verbose = 1)

In [ ]:
f1, ax1 = plt.subplots()
ax1.plot([None] + plot_Inception.history['accuracy'], 'o-')
ax1.plot([None] + plot_Inception.history['val_accuracy'], 'x-')
# Plot legend and use the best location automatically: loc = 0.
ax1.legend(['Train acc', 'Validation acc'], loc = 0)
ax1.set_title('Training/Validation acc per Epoch')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('acc')

# **Keras Inception V3 (Transfer Learning)**

In [ ]:
input_shape = (75, 75, 3)
X_input = Input(input_shape)

inceptionModeltf = Sequential()
inceptionModeltf.add(InceptionV3(include_top = False, weights = 'imagenet', input_tensor = X_input, input_shape = input_shape))
inceptionModeltf.add(GlobalAveragePooling2D())
inceptionModeltf.add(Dense(10, activation = 'softmax'))
inceptionModeltf.compile(optimizer= 'adam', loss='categorical_crossentropy', metrics=['accuracy'])
inceptionModeltf.layers[0].trainable = False
inceptionModeltf.summary()

In [ ]:
plot_pretrained_Inception = inceptionModeltf.fit(InceptionDatagen.flow(inception_xtrain, y_train, batch_size=128),
                        steps_per_epoch=len(x_train) / 128, epochs = 20, validation_data=(inception_xtest, y_test), verbose = 1)

In [ ]:
f1, ax1 = plt.subplots()
ax1.plot([None] + plot_pretrained_Inception.history['accuracy'], 'o-')
ax1.plot([None] + plot_pretrained_Inception.history['val_accuracy'], 'x-')
# Plot legend and use the best location automatically: loc = 0.
ax1.legend(['Train acc', 'Validation acc'], loc = 0)
ax1.set_title('Training/Validation acc per Epoch')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('acc')

# **AlexNet**

In [ ]:
def AlexNetModel():
  #Instantiate an empty model
  AlexNet = Sequential()

  # LAYER 1
  AlexNet.add(layers.Conv2D(96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=(32,32,3), padding="same"))
  ## pooling
  AlexNet.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same"))
  AlexNet.add(layers.BatchNormalization())

  # LAYER 2
  AlexNet.add(layers.ZeroPadding2D(padding=(2, 2)))
  AlexNet.add(layers.Conv2D(256, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding="same"))
  ## pooling
  AlexNet.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same"))
  AlexNet.add(layers.BatchNormalization())

  # LAYER 3
  AlexNet.add(layers.ZeroPadding2D(padding=(1, 1)))
  AlexNet.add(layers.Conv2D(384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"))

  # LAYER 4
  AlexNet.add(layers.ZeroPadding2D(padding=(1, 1)))
  AlexNet.add(layers.Conv2D(384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"))

  # LAYER 5
  AlexNet.add(layers.ZeroPadding2D(padding=(1, 1)))
  AlexNet.add(layers.Conv2D(256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"))
  ## pooling
  AlexNet.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same"))

  # Flatten the CNN output so that we can connect it with fully connected layers
  AlexNet.add(layers.Flatten())

  # FC6 Fully Connected Layer
  AlexNet.add(layers.Dense(4096, activation='relu'))
  AlexNet.add(layers.Dropout(0.5))

  # FC7 Fully Connected Layer
  AlexNet.add(layers.Dense(4096, activation='relu'))
  AlexNet.add(layers.Dropout(0.5))

  # Output Layer with softmax activation
  AlexNet.add(layers.Dense(10, activation="softmax"))

  # Compile the model
  AlexNet.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=["accuracy"])

  return AlexNet

In [ ]:
AlexNet = AlexNetModel()
AlexNet.summary()

In [ ]:
alexnet = AlexNet.fit_generator(datagen.flow(x_train, y_train, batch_size=128), 
                           steps_per_epoch=len(x_train) / 128, epochs = 50, validation_data=(x_test, y_test), verbose = 1)

In [ ]:
f1, ax1 = plt.subplots()
ax1.plot([None] + alexnet.history['accuracy'], 'o-')
ax1.plot([None] + alexnet.history['val_accuracy'], 'x-')
# Plot legend and use the best location automatically: loc = 0.
ax1.legend(['Train acc', 'Validation acc'], loc = 0)
ax1.set_title('Training/Validation acc per Epoch')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('acc')

# **Resnet (Transfer Learning)**

In [ ]:
ResNet = Sequential()

ResNet.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
ResNet.add(Flatten())
ResNet.add(BatchNormalization())
ResNet.add(Dense(2048, activation='relu'))
ResNet.add(BatchNormalization())
ResNet.add(Dense(1024, activation='relu'))
ResNet.add(BatchNormalization())
ResNet.add(Dense(10, activation='softmax'))

ResNet.layers[0].trainable = False

ResNet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
ResNet.summary()

In [ ]:
resnet_pre = ResNet.fit(datagen.flow(x_train, y_train, batch_size=128),
                    steps_per_epoch=len(x_train) / 128, epochs = 50, validation_data=(x_test, y_test), verbose = 1)

In [ ]:
f1, ax1 = plt.subplots()
ax1.plot([None] + resnet_pre.history['accuracy'], 'o-')
ax1.plot([None] + resnet_pre.history['val_accuracy'], 'x-')
# Plot legend and use the best location automatically: loc = 0.
ax1.legend(['Train acc', 'Validation acc'], loc = 0)
ax1.set_title('Training/Validation acc per Epoch')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('acc')

# **Resnet (No Transfer Learning)**

In [ ]:
ResNet_NonPre = Sequential()

ResNet_NonPre.add(ResNet50(include_top=False, pooling='avg', weights=None))
ResNet_NonPre.add(Flatten())
ResNet_NonPre.add(BatchNormalization())
ResNet_NonPre.add(Dense(2048, activation='relu'))
ResNet_NonPre.add(BatchNormalization())
ResNet_NonPre.add(Dense(1024, activation='relu'))
ResNet_NonPre.add(BatchNormalization())
ResNet_NonPre.add(Dense(10, activation='softmax'))

ResNet_NonPre.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
ResNet_NonPre.summary()

In [ ]:
plot_nonpre_resnet = ResNet_NonPre.fit(datagen.flow(x_train, y_train, batch_size=128),
                    steps_per_epoch=len(x_train) / 128, epochs = 50, validation_data=(x_test, y_test), verbose = 1)

In [ ]:
f1, ax1 = plt.subplots()
ax1.plot([None] + plot_nonpre_resnet.history['accuracy'], 'o-')
ax1.plot([None] + plot_nonpre_resnet.history['val_accuracy'], 'x-')
# Plot legend and use the best location automatically: loc = 0.
ax1.legend(['Train acc', 'Validation acc'], loc = 0)
ax1.set_title('Training/Validation acc per Epoch')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('acc')